In [0]:
from google.colab import files
uploaded = files.upload()

###1) Pixibo influenced purchases -A size recommendation was made and the product was purchased

In [0]:
#reading data_science_exchanges_returns.csv renamed as pixibo1.xlsx
import pandas as pd
return_df = pd.read_excel('pixibo1.xlsx', 'Sheet1')
return_df.columns = return_df.columns.str.strip().str.replace(' ', '_') #removing space from column names



In [0]:
return_df.shape

(91242, 8)

In [0]:
#reading data_science_test_conversions.json
df_conv = pd.read_json('data_science_test_conversions.json', lines = True)

In [0]:
df_conv.head()

,_id,_index,_score,_source,_type
0,8a8b6404-e312-4bef-98cd-df3637d09b2f,tracking_prod,7.577892,"{'uid': 'unknown', 'amount': '1394', 'clientId...",conversion
1,a98d0ef4-bb5b-48d5-b01d-2cc9b2279a7a,tracking_prod,7.577892,{'uid': '0fc636a4-a03f-493d-8122-64f98c12e6df'...,conversion
2,553f06bf-d9a2-4aa7-9ff7-a2959b884994,tracking_prod,7.577892,{'uid': '3c3c50b0-99a2-494e-8228-b559eafccc02'...,conversion
3,420fc2c3-0ce4-4d21-ab88-cff69d4a3643,tracking_prod,7.577892,"{'uid': 'unknown', 'amount': '1299', 'clientId...",conversion
4,44e993a5-c4f4-4cfc-8bd3-242e8b418893,tracking_prod,7.577892,"{'uid': 'unknown', 'amount': '2091.56', 'clien...",conversion


In [0]:
#fetching transaction details from data_science_test_conversions.json which are present in _source column
details = pd.read_json((df_conv['_source']).to_json(), orient = 'index')
details.head()
details.shape

(62482, 7)

In [0]:
#filtering details for february month
transaction = details[(details['timestamp'] >= '2019-02-01 00:00:00.000') & (details['timestamp'] < '2019-03-01 00:00:00.000')]
transaction.shape

(4705, 7)

In [0]:
transaction.head()

,amount,clientId,currency,products,timestamp,transaction,uid
5185,1143.12,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-07 08:29:55.536,215125962,unknown
5186,1150.00,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'UK 10', 'price': '11...",2019-02-07 08:49:49.141,215325962,f6fb8346-1b77-4e7f-a631-cfb4b571ceb6
5187,1996.00,ubu8zhxrh8hg,PHP,"[{'quantity': 2, 'size': 'International XXXL',...",2019-02-07 10:04:50.854,279965962,c5c23898-6998-4ae3-b4e9-9f4477ecd6bf
5188,3650.00,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International One Si...",2019-02-07 17:41:34.990,267385962,a695c21f-e7ea-4a80-8719-a658a9af045e
5189,1750.00,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International One Si...",2019-02-08 01:31:22.243,273455962,unknown


In [0]:
# reading data_science_test_events.json
import json
import urlopen
import urllib.request
import xlsxwriter
import pandas as pd
user, product, recommendation, timestamp = ([] for i in range(4))
f = urllib.request.urlopen('https://s3-us-west-2.amazonaws.com/miscka/datascience/data_science_test_events.json')
for line in f:
  data = json.loads(line)
  if (data['_source']['timestamp'] >= '2019-02-01T00:00:00.000Z' and data['_source']['timestamp'] < '2019-03-01T00:00:00.000Z'):
      if (data['_source']['eventType'] == 'size' and data['_source']['statusCode'] == 200):
        uid = data['_source']['uid']
        user.append(uid)
        pid = data['_source']['skuId']
        product.append(pid)  
        response = (data['_source']['response']['fys'])
        recommendation.append(response)
        time = data['_source']['timestamp']
        timestamp.append(time)
events_df = pd.DataFrame({'user':user, 'product':product, 'recommended_size': recommendation, 'time': timestamp})



In [0]:
#product column has products fetched from data_science_test_events.json for eventType = size
events_df.head()

,user,product,recommended_size,time
0,31e4db3d-6377-4d00-b2a7-b8243572db0e,cb29daa678e645gs,"[{'size': 'INT L', 'confidence': 2, 'bust': 3,...",2019-02-01T02:00:37.331Z
1,e45dba3f-2ea6-4ad7-84eb-3b7d625d2447,d993caa823c57cgs,"[{'size': 'INT S', 'confidence': 3, 'bust': 5,...",2019-02-01T03:31:48.672Z
2,4e7edc56-1b63-488b-a9c2-8628daab1e32,56bafaa4ca5e40gs,"[{'size': 'INT XL', 'confidence': 1, 'bust': 5...",2019-02-01T04:55:06.757Z
3,1f41fc4f-85d3-459b-b4f2-90bcf60a9c12,905e3aadbb0671gs,"[{'size': 'INT XS', 'confidence': 2, 'bust': 2...",2019-02-01T07:18:27.504Z
4,62658451-3091-4402-b663-6da4bf9291da,348a5aac44fd0ags,"[{'size': 'INT M', 'confidence': 3, 'bust': 5,...",2019-02-01T07:56:24.721Z


In [0]:
#renaming user column to uid so that both transaction and events_df dataframes can be merged on uid column
events_df.columns = ['uid', 'product', 'recommended_size', 'time']

In [0]:
events_df.head()

,uid,product,recommended_size,time
0,31e4db3d-6377-4d00-b2a7-b8243572db0e,cb29daa678e645gs,"[{'size': 'INT L', 'confidence': 2, 'bust': 3,...",2019-02-01T02:00:37.331Z
1,e45dba3f-2ea6-4ad7-84eb-3b7d625d2447,d993caa823c57cgs,"[{'size': 'INT S', 'confidence': 3, 'bust': 5,...",2019-02-01T03:31:48.672Z
2,4e7edc56-1b63-488b-a9c2-8628daab1e32,56bafaa4ca5e40gs,"[{'size': 'INT XL', 'confidence': 1, 'bust': 5...",2019-02-01T04:55:06.757Z
3,1f41fc4f-85d3-459b-b4f2-90bcf60a9c12,905e3aadbb0671gs,"[{'size': 'INT XS', 'confidence': 2, 'bust': 2...",2019-02-01T07:18:27.504Z
4,62658451-3091-4402-b663-6da4bf9291da,348a5aac44fd0ags,"[{'size': 'INT M', 'confidence': 3, 'bust': 5,...",2019-02-01T07:56:24.721Z


In [0]:
#merging events and transactions dataframes on userid
mergeddf = pd.merge(transaction, events_df, on=['uid'], how='inner')


In [0]:
#dropping columns from the merged dataframe
mergeddf.drop(['amount', 'clientId', 'currency'], axis = 1, inplace = True)

In [0]:
mergeddf.head()

,products,timestamp,transaction,uid,product,recommended_size,time
0,"[{'quantity': 1, 'size': 'US 9', 'price': '180...",2019-02-07 10:34:59.529,283865962,9ada6412-1878-4e6b-8d8d-2f5da37d4e51,64730aa75436eegs,"[{'size': 'INT S', 'confidence': 4, 'bust': 2,...",2019-02-15T04:11:18.908Z
1,"[{'quantity': 1, 'size': 'US 9', 'price': '292...",2019-02-04 07:04:04.817,294479962,9ada6412-1878-4e6b-8d8d-2f5da37d4e51,64730aa75436eegs,"[{'size': 'INT S', 'confidence': 4, 'bust': 2,...",2019-02-15T04:11:18.908Z
2,"[{'quantity': 1, 'size': 'Waist Size 30 in', '...",2019-02-09 10:00:16.909,284431962,b00d784b-99d8-462c-8456-53e549c8ad93,ad2c0aa344eeafgs,"[{'size': 'UK 30S', 'confidence': 3, 'bust': 0...",2019-02-09T09:39:00.238Z
3,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-01 02:29:41.281,236742962,3528be91-d6a2-4ec6-b26d-076dcc8ee15e,a00f7aa5e23e13gs,"[{'size': 'INT S', 'confidence': 4, 'bust': 0,...",2019-02-01T02:22:00.260Z
4,"[{'quantity': 1, 'size': 'International M', 'p...",2019-02-01 06:29:45.591,296962962,277cc0f6-a9b6-45f0-aa41-69509407322e,1076caa8f82abegs,"[{'size': 'INT S', 'confidence': 2, 'bust': 1,...",2019-02-20T06:22:17.467Z


In [0]:
writer = pd.ExcelWriter('merged.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
mergeddf.to_excel(writer,'Sheet1')
writer.save()

In [0]:
#fetching product ids from products column in mergeddf
list = []
for d in mergeddf.products:
  if(len(d) > 1):
    products = []
    [products.append(x.get('sku')) for x in d]
    list.append(products)
  else:
    [list.append(x.get('sku')) for x in d]
print (list)

In [0]:
#adding new column to mergeddf
# trans_productid contains the product ids exracted from products column 
mergeddf['trans_productid'] = list

In [0]:
mergeddf.head()


,products,timestamp,transaction,uid,product,recommended_size,time,trans_productid
0,"[{'quantity': 1, 'size': 'US 9', 'price': '180...",2019-02-07 10:34:59.529,283865962,9ada6412-1878-4e6b-8d8d-2f5da37d4e51,64730aa75436eegs,"[{'size': 'INT S', 'confidence': 4, 'bust': 2,...",2019-02-15T04:11:18.908Z,A7902SH3077F68GS
1,"[{'quantity': 1, 'size': 'US 9', 'price': '292...",2019-02-04 07:04:04.817,294479962,9ada6412-1878-4e6b-8d8d-2f5da37d4e51,64730aa75436eegs,"[{'size': 'INT S', 'confidence': 4, 'bust': 2,...",2019-02-15T04:11:18.908Z,68312SHE6BC64BGS
2,"[{'quantity': 1, 'size': 'Waist Size 30 in', '...",2019-02-09 10:00:16.909,284431962,b00d784b-99d8-462c-8456-53e549c8ad93,ad2c0aa344eeafgs,"[{'size': 'UK 30S', 'confidence': 3, 'bust': 0...",2019-02-09T09:39:00.238Z,"[64D8CAA1B69D83GS, 769ECAADC6901AGS, B08C3AAFC..."
3,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-01 02:29:41.281,236742962,3528be91-d6a2-4ec6-b26d-076dcc8ee15e,a00f7aa5e23e13gs,"[{'size': 'INT S', 'confidence': 4, 'bust': 0,...",2019-02-01T02:22:00.260Z,"[994DBAA2817BC8GS, A00F7AA5E23E13GS, 61686AAA4..."
4,"[{'quantity': 1, 'size': 'International M', 'p...",2019-02-01 06:29:45.591,296962962,277cc0f6-a9b6-45f0-aa41-69509407322e,1076caa8f82abegs,"[{'size': 'INT S', 'confidence': 2, 'bust': 1,...",2019-02-20T06:22:17.467Z,"[1EE8BAA28432A4GS, D254AAAB83182CGS]"


In [0]:
mergeddf.shape

(11963, 8)

In [0]:
mergeddf['product'] = mergeddf['product'].str.upper()

In [0]:
#copying rows to new dataframe in which product id is in trans_productid
new_df = pd.DataFrame(columns=mergeddf.columns)
for i in mergeddf.index:
  if (mergeddf['product'][i] in mergeddf['trans_productid'][i]):
    new_df = new_df.append(mergeddf.loc[i], ignore_index = True)

In [0]:
new_df

,products,timestamp,transaction,uid,product,recommended_size,time,trans_productid
0,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-01 02:29:41.281,236742962,3528be91-d6a2-4ec6-b26d-076dcc8ee15e,A00F7AA5E23E13GS,"[{'size': 'INT S', 'confidence': 4, 'bust': 0,...",2019-02-01T02:22:00.260Z,"[994DBAA2817BC8GS, A00F7AA5E23E13GS, 61686AAA4..."
1,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-01T15:39:50.702Z,"[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
2,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-01T15:41:32.259Z,"[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
3,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-03T07:00:07.605Z,"[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
4,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-17T00:25:07.689Z,"[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
5,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-01T15:44:13.646Z,"[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
6,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-01T15:44:32.208Z,"[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
7,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-02T00:38:02.997Z,"[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
8,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-18 14:27:37.607,251985862,0c349735-c293-4731-ae90-451b8a1da885,4E0D7AAB686A0BGS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-06T12:21:25.174Z,"[4E0D7AAB686A0BGS, 39FFDAA3A64B28GS, C71A3AA99..."
9,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-18 14:27:37.607,251985862,0c349735-c293-4731-ae90-451b8a1da885,4E0D7AAB686A0BGS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...",2019-02-16T15:28:59.347Z,"[4E0D7AAB686A0BGS, 39FFDAA3A64B28GS, C71A3AA99..."


In [0]:
new_df.drop('time', axis = 1, inplace = True )

In [0]:
new_df.head()

,products,timestamp,transaction,uid,product,recommended_size,trans_productid
0,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-01 02:29:41.281,236742962,3528be91-d6a2-4ec6-b26d-076dcc8ee15e,A00F7AA5E23E13GS,"[{'size': 'INT S', 'confidence': 4, 'bust': 0,...","[994DBAA2817BC8GS, A00F7AA5E23E13GS, 61686AAA4..."
1,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...","[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
2,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...","[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
3,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...","[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."
4,"[{'quantity': 1, 'size': 'International L', 'p...",2019-02-03 07:02:11.840,221356962,0c349735-c293-4731-ae90-451b8a1da885,BAB9CAA0751876GS,"[{'size': 'INT S', 'confidence': 2, 'bust': 5,...","[CH672AA0KGM2PH, BAB9CAA0751876GS, 759D2AA21B2..."


In [0]:
writer = pd.ExcelWriter('total.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
new_df.to_excel(writer,'Sheet1')
writer.save()

In [0]:
mergeddf.shape

(11963, 8)

In [0]:
#duplicates removed from total.xlsx
total_df = pd.read_excel('total.xlsx', 'Sheet1')
total_df.shape

(117, 7)

In [0]:
#order numbers which are common to both transaction and returns dataframes
s = set(total_df['transaction']).intersection(set(return_df['Order_nr']))
print ("total number of returns " + " "+ str(len(s)))

total number of returns  21


###A size recommendation was made and the product was purchased 
number of products sold = 117
total returns = 21
size & fit returns rate = 117/21

In [0]:
117/21

5.571428571428571

In [0]:
total_df.shape

(117, 7)

###34 rows containing "recommended = false"" only were removed from totals.xlsx
###** International was replaced with INT**

In [0]:
import pandas as pd
new_df = pd.read_excel('total_new.xlsx')
new_df.shape

(83, 7)

In [0]:
new_df.head()

,products,timestamp,transaction,uid,product,recommended_size,trans_productid
0,"[{'quantity': 1, 'size': 'INT S', 'price': '47...",2019-02-01 02:29:41.281,236742962,3528be91-d6a2-4ec6-b26d-076dcc8ee15e,A00F7AA5E23E13GS,"[{'size': 'INT S', 'confidence': 4, 'bust': 0,...","['994DBAA2817BC8GS', 'A00F7AA5E23E13GS', '6168..."
1,"[{'quantity': 1, 'size': 'Waist x Length W2926...",2019-02-06 19:18:41.887,251917962,0c349735-c293-4731-ae90-451b8a1da885,9C659AA4AED569GS,"[{'size': '28', 'confidence': 3, 'bust': 0, 'w...",9C659AA4AED569GS
2,"[{'quantity': 1, 'size': 'INT XL', 'price': '7...",2019-02-11 21:03:28.462,221114862,65e57b99-d3c6-433d-93f0-6550f721260e,625B4AA0F3327BGS,"[{'size': 'INT L', 'confidence': 3, 'bust': 2,...",625B4AA0F3327BGS
3,"[{'quantity': 1, 'size': 'INT S', 'price': '79...",2019-02-21 04:54:33.834,244983862,282a5d73-a9f3-41ac-913c-c289586bba0b,1F3CDAA6CE7A94GS,"[{'size': 'UK 8', 'confidence': 2, 'bust': 1, ...","['B2957AAC19B6E2GS', '1F3CDAA6CE7A94GS']"
4,"[{'quantity': 1, 'size': 'INT M', 'price': '66...",2019-02-06 10:56:03.886,291477962,385d88cf-4c07-4238-afae-2e30e011be46,B0420AA4264AEBGS,"[{'size': 'INT M', 'confidence': 4, 'bust': 0,...","['0EF77AAE6D1367GS', '47158AAD1878D9GS', 'B042..."


In [0]:
purchased_size = []
i = 0
import ast
for d in new_df.products:
  a = ast.literal_eval(d) #converting string(from products column) into dictionary
  for x in a:
    if (x.get('sku') == new_df['product'][i]):
      purchased_size.append(x.get('size'))
      i += 1 
      break

In [0]:
print(purchased_size)

['INT M', 'Waist x Length W2926', 'INT XL', 'UK 8', 'INT L', 'INT L', 'INT L', 'US 4', 'UK 12', 'UK 8', 'INT S', 'INT S', 'INT XL', 'INT L', 'INT M', 'INT S', 'INT M', 'INT M', 'INT M', 'INT S', 'INT XL', 'INT XL', 'INT M', 'INT L', 'EU 34', 'INT M', 'INT M', 'INT S', 'INT S', 'INT S', 'INT S', 'INT L', 'INT M', 'INT S', 'INT S', 'EU 36', 'INT XS', 'INT S', 'INT M', 'INT S', 'INT M', 'INT XL', 'INT XL', 'INT L', 'INT L', 'INT L', 'INT L', 'INT XL', 'INT XS', 'INT M', 'INT M', 'INT M', 'INT L', 'INT L', 'INT M', 'INT XS/S', 'INT M', 'INT L', 'INT S', 'INT S', 'INT S', 'EU 36', 'INT XXS', 'AUS 8', 'INT XXS', 'INT S', 'INT M', 'INT M', 'INT S', 'INT M', 'INT M', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT S', 'EU 38', 'INT M', 'INT M', 'INT XS', 'UK 14', 'EU 38']


In [0]:

import ast
recommended_size = []
for d in new_df.recommended_size:
  a = ast.literal_eval(d)
  for x in a:
    if(x.get('recommended') == True):
          recommended_size.append(x.get('size'))

print (recommended_size)        

['INT M', '29', 'INT XL', 'UK 10', 'INT L', 'INT XL', 'INT XL', 'UK 12', 'UK 12', 'UK 8', 'INT XS', 'INT S', 'INT XL', 'INT XL', 'INT M', 'INT S', 'INT S', 'INT S', 'INT M', 'INT S', 'INT L', 'INT XL', 'INT M', 'INT L', 'EU 34', 'INT L', 'INT M', 'INT L', 'INT S', 'INT M', 'INT S', 'INT M', 'INT M', 'INT XS', 'INT XS', 'EU 36', 'INT XS', 'INT S', 'INT M', 'INT L', 'INT M', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT S', 'INT S', 'INT L', 'INT L', 'INT L', 'INT L', 'INT M', 'UK 12', 'INT M', 'INT L', 'INT L', 'INT S', 'INT S', 'EU 36', 'INT XS', 'AU 8', 'INT XS', 'INT XS', 'INT XS', 'INT XS', 'INT S', 'INT M', 'INT M', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT XL', 'INT S', 'EU 38', 'INT M', 'INT S', 'INT XS', 'UK 14', 'EU 38']


In [0]:
pix_df = pd.DataFrame(columns=new_df.columns) #Purchased Size = Pixibo Recommended Size (Optional)
notpix_df = pd.DataFrame(columns=new_df.columns)#Purchased Size != PixiboRecommended Size (Optional)
for i in range(len(recommended_size)):
  if(recommended_size[i] in purchased_size[i]):
    pix_df = pix_df.append(new_df.loc[i], ignore_index = True)
  else:
    notpix_df = notpix_df.append(new_df.loc[i], ignore_index = True)
  

In [0]:
pix_df.shape

(50, 7)

In [0]:
!pip install xlsxwriter

In [0]:
import xlsxwriter
writer = pd.ExcelWriter('pixibo(1B).xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
notpix_df.to_excel(writer,'Sheet1')
writer.save()

In [0]:
notpix_df.shape

In [0]:
#Purchased Size = Pixibo Recommended Size (Optional)
#order numbers which are common to both transaction and returns dataframes
s = set(pix_df['transaction']).intersection(set(return_df['Order_nr']))
print ("total number of returns " + " "+ str(len(s)))

total number of returns  5


In [0]:
#Purchased Size != Pixibo Recommended Size (Optional)
#order numbers which are common to both transaction and returns dataframes
s = set(notpix_df['transaction']).intersection(set(return_df['Order_nr']))
print ("total number of returns " + " "+ str(len(s)))

total number of returns  7


###Purchased Size = Pixibo Recommended Size (Optional)

number of products sold = 50 total returns = 5 size & fit returns rate = 50/5

###Purchased Size != Pixibo Recommended Size (Optional)
number of products sold = 33 total returns = 7 size & fit returns rate = 33/7

